# 📊 Datasheet Analyzer

**AI를 활용한 데이터시트 분석 도구입니다. 코딩 지식 없이도 쉽게 사용할 수 있습니다.**

---

## 🚀 시작하기 전에

**[필수] GPU 설정:** 
1. 상단 메뉴 → **런타임** → **런타임 유형 변경**
2. **하드웨어 가속기: GPU**로 설정
3. **저장** 클릭

## 📋 사용 방법
아래 셀들을 **순서대로 실행**만 하면 됩니다. (각 셀의 ▶️ 버튼을 클릭)

---

In [ ]:
#@title ✅ 1단계: GPU 설정 확인 { display-mode: "form" }
#@markdown GPU가 제대로 설정되었는지 확인합니다.

import torch
print("🔍 시스템 확인 중...\n")

# GPU 확인
!nvidia-smi --query-gpu=name,memory.total --format=csv,noheader

# PyTorch GPU 확인
print(f"\n📍 PyTorch 버전: {torch.__version__}")
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    print(f"✅ GPU 사용 가능: {gpu_name}")
    print("🎉 준비 완료! 다음 단계로 진행하세요.")
else:
    print("❌ GPU를 찾을 수 없습니다.")
    print("⚠️ 상단 메뉴에서 런타임 → 런타임 유형 변경 → GPU로 설정하세요.")

In [ ]:
#@title 📥 2단계: 프로그램 다운로드 { display-mode: "form" }
#@markdown GitHub에서 최신 버전을 다운로드합니다.

import os
from pathlib import Path

print("📥 Datasheet Analyzer 다운로드 중...")

# 이미 다운로드된 경우 삭제 후 새로 다운로드
if os.path.exists('autodatasheet'):
    !rm -rf autodatasheet
    print("🔄 기존 파일 정리 완료")

# Repository 복제
!git clone https://github.com/doyoung42/autodatasheet.git
%cd autodatasheet

print("✅ 다운로드 완료!")
print("📁 프로젝트 구조:")
!find . -maxdepth 2 -type d | head -10

In [ ]:
#@title 🔧 3단계: 필요한 라이브러리 설치 { display-mode: "form" }
#@markdown 프로그램 실행에 필요한 라이브러리들을 설치합니다. (1-2분 소요)

print("🔧 필요한 라이브러리 설치 중...")
print("⏰ 약 1-2분 정도 소요됩니다. 잠시만 기다려주세요.")

# 기본 라이브러리 설치
!pip install -r requirements.txt -q

# Colab 전용 추가 라이브러리
!pip install streamlit-option-menu -q

print("✅ 모든 라이브러리 설치 완료!")
print("🎯 다음 단계에서 API 토큰을 설정하세요.")

In [ ]:
#@title 🔑 4단계: API 토큰 설정 { display-mode: "form" }
#@markdown 사용하실 AI 서비스의 API 토큰을 입력하세요. (HuggingFace 입력 필수)

huggingface_token = "" #@param {type:"string"}
openai_token = "" #@param {type:"string"}
anthropic_token = "" #@param {type:"string"}

# 시스템 경로 설정
import sys
import os
from pathlib import Path

current_dir = Path(os.getcwd())
sys.path.append(str(current_dir))
sys.path.append(str(current_dir / 'src'))

src_dir = current_dir / 'src'
if src_dir.exists():
    sys.path.append(str(src_dir))
    for subdir in ['config', 'models', 'utils']:
        subdir_path = src_dir / subdir
        if subdir_path.exists():
            sys.path.append(str(subdir_path))

# 토큰 저장
try:
    from config.token_manager import TokenManager
    token_manager = TokenManager()
    
    saved_tokens = []
    
    if huggingface_token.strip():
        token_manager.set_token('huggingface', huggingface_token.strip())
        saved_tokens.append('HuggingFace')
    
    if openai_token.strip():
        token_manager.set_token('openai', openai_token.strip())
        saved_tokens.append('OpenAI')
    
    if anthropic_token.strip():
        token_manager.set_token('anthropic', anthropic_token.strip())
        saved_tokens.append('Anthropic')
    
    if saved_tokens:
        print(f"✅ {', '.join(saved_tokens)} API 토큰이 저장되었습니다!")
        print("🎉 토큰 설정 완료! 다음 단계로 진행하세요.")
    else:
        print("⚠️ 입력된 토큰이 없습니다.")
        print("💡 최소 1개 이상의 API 토큰을 입력해주세요.")
        
        print("\n📚 API 토큰 발급 방법:")
        print("• HuggingFace: https://huggingface.co/settings/tokens")
        print("• OpenAI: https://platform.openai.com/api-keys")
        print("• Anthropic: https://console.anthropic.com/")
        
except Exception as e:
    print(f"❌ 토큰 저장 중 오류가 발생했습니다: {str(e)}")
    print("🔧 다시 시도하거나 개발자에게 문의하세요.")

In [ ]:
#@title 🚀 5단계: 웹 애플리케이션 실행 { display-mode: "form" }
#@markdown 웹 브라우저에서 사용할 수 있는 애플리케이션을 실행합니다.

import subprocess
import threading
import time
import os
from pathlib import Path

print("🚀 Datasheet Analyzer 웹 애플리케이션을 시작합니다...")
print("⏰ 약 10초 정도 소요됩니다.")

def run_streamlit():
    """백그라운드에서 Streamlit 실행"""
    try:
        env = os.environ.copy()
        env.update({
            'STREAMLIT_SERVER_HEADLESS': 'true',
            'STREAMLIT_SERVER_PORT': '8501',
            'STREAMLIT_SERVER_ADDRESS': '0.0.0.0',
            'STREAMLIT_BROWSER_SERVER_ADDRESS': '0.0.0.0',
            'STREAMLIT_SERVER_MAX_UPLOAD_SIZE': '200',
            'STREAMLIT_SERVER_ENABLE_CORS': 'false',
            'STREAMLIT_SERVER_ENABLE_XSRF_PROTECTION': 'false',
            'PYTHONPATH': f"{Path.cwd()}:{Path.cwd()}/src"
        })
        
        subprocess.run([
            'streamlit', 'run', 'src/app/streamlit_app.py',
            '--server.port=8501',
            '--server.address=0.0.0.0',
            '--server.maxUploadSize=200',
            '--server.enableCORS=false',
            '--server.enableXsrfProtection=false'
        ], env=env, capture_output=True)
        
    except Exception as e:
        print(f"❌ 애플리케이션 실행 중 오류: {str(e)}")

# 백그라운드에서 실행
thread = threading.Thread(target=run_streamlit, daemon=True)
thread.start()

# 서버 시작 대기
print("⏳ 서버 준비 중...")
time.sleep(10)

# 웹 URL 생성
try:
    from google.colab.output import eval_js
    url = eval_js('google.colab.kernel.proxyPort(8501)')
    
    print("\n" + "="*50)
    print("🎉 애플리케이션이 준비되었습니다!")
    print("\n🔗 아래 링크를 클릭하여 사용하세요:")
    print(f"📱 {url}")
    print("="*50)
    print("\n💡 사용 팁:")
    print("• 링크를 새 탭에서 열어서 사용하세요")
    print("• 파일 업로드는 최대 200MB까지 가능합니다")
    
except Exception as e:
    print(f"❌ URL 생성 실패: {str(e)}")
    print("🔧 수동으로 포트 8501에 접속해보세요.")

---

## 🛠️ 문제 해결

**애플리케이션이 실행되지 않는 경우:**

1. **GPU 설정 확인**: 런타임 → 런타임 유형 변경 → GPU 설정
2. **세션 재시작**: 런타임 → 세션 재시작 후 처음부터 다시 실행
3. **토큰 확인**: API 토큰이 올바르게 입력되었는지 확인

**사용 중 문제가 발생하는 경우:**
- 브라우저를 새로고침하세요
- 파일 크기가 200MB를 초과하지 않는지 확인하세요
- 네트워크 연결 상태를 확인하세요

---

## 📞 지원

추가 도움이 필요하시면 GitHub 이슈를 등록해주세요:
🔗 https://github.com/doyoung42/autodatasheet/issues
